Rename a bunch of files using Git and renamer?



In [ ]:
var renamer = require('renamer');

function renameUsingGit(gitRoot, match, find, replace) {
    var files = renamer.expand(path.join(gitRoot, match));
    var results = renamer.replace({
        files: files.filesAndDirs,
        find: find,
        replace: replace
    });
    return renamer.dryRun(results).list.map(r => {
        // rename with git instead
        var output = execSync('git '
            + '--work-tree=' + JSON.stringify(gitRoot)
            + ' --git-dir=' + JSON.stringify(path.join(gitRoot, '.git'))
            + ' mv ' + JSON.stringify(r.before)
            + ' ' + JSON.stringify(r.after));
        return output.toString() || r.after;
    });
};
renameUsingGit;


test github renamer?

In [ ]:
// test it
// rename all scss files to less
//renameUsingGit(
//    '/Users/briancullinan/Documents/studysauce4',
//    'src/**/*.css',
//    '.css',
//    '.scss');


json git tree?


In [ ]:
var execSync = require('child_process').execSync;
/*var {
    execCmd
} = importer.import([
    'spawn child process',
]);*/


function gitTree(project) {
    console.log('creating tree "' + project + '"');
    var branches = execSync('git branch --list --all ', {cwd: project}).toString()
        .trim().split(/\s*\n\s*\**\s*/ig)
        .map(b => b.replace('remotes/', '').trim());

    // reverse git so we can add merges to the "graph" after the branch tree is constructed
    var branch = execSync('git log --pretty=format:"%ct %d %h %p" '
        + '--reverse --branches --tags ', {cwd: project}).toString().split(/\s*\n\s*/ig);
    var all = []; //execSync('git log --pretty=format:"%h %p" --reverse --all').toString().split(/\s*\n\s*/ig);

    var nodes = [];
    var edges = [];
    branch.forEach(b => {
        var commits = ((/[0-9]+\s+\(.*?\)\s*(.+)/ig).exec(b)
            || (/[0-9]+\s+(.+)/ig).exec(b) || [b, b.trim()])[1].trim().split(/\s+/ig);
        commits.forEach(r => {
            if (nodes.filter(d => d.name == r).length == 0) {
                nodes[nodes.length] = {
                    name: r,
                    index: nodes.length,
                    branch: ((/[0-9]+\s+\((.*?)\)/ig).exec(b) || [b, ''])[1],
                    time: (/([0-9]+)\s+/ig).exec(b)[1]
                }
            }
        });
        // don't include merges in tree
        if (commits.length == 3) {
            commits = [commits[0], commits[1]];
        }
        if (commits.length == 2) {
            edges = edges.concat(commits.slice(1)
                // check all existing sources to test for merge
                .map((c, i) => {
                    return {source: c, target: commits[i]}; // -1 because of slice(1)
                }))
        }
    });

    return formatNodes(nodes, edges.reverse());
}
module.exports = gitTree;



tip git tree?

just the tip?

source tree?



In [ ]:
var importer = require('../Core');
var _ = require('underscore');
var fs = require('fs');
var path = require('path');
var {
    formatNodes, gitTree, displayBranches,
    cloneProject, execCmd
} = importer.import([
    'spawn child process',
    'd3.ipynb[format d3 tree]',
    'json git tree',
    'd3.ipynb[display d3 tree]',
    'git project directory'
]);

// TODO: convert this to bash
function getResetCommit(branch, parent, project) {
    branch = branch.replace('HEAD -> ', '').trim();
    parent = parent.replace('HEAD -> ', '').trim();
    return execCmd(`
git checkout ${branch}
git reset --mixed ${parent}
git add -A
git commit -m "auto reset ${branch}"
`, {cwd: project});
}

var svgs = [];
function resetAllBranches() {
    var children = [];
    var nodeTree = _.sortBy(gitTree(project), a => a.time);
    var promises = [];
    for (const n of nodeTree) {
        var current = n;
        var parentCount = 0;
        while (typeof current.parent !== 'undefined'
            && current.parent.branch === current.branch) {
            current = current.parent;
            parentCount++;
        }
        if (parentCount > 0 && typeof current.parent !== 'undefined') {
            // TODO: keep track of the parent so the squashed branch can be merged on to the newly squashed parent
            promises.push(((current, n) => resolve => {
                if (typeof n.children !== 'undefined') {
                    children.push(n);
                }
                return getResetCommit(n.branch, current.parent.name, project)
                    .then(() => {
                        var nodeTree = _.sortBy(gitTree(project), a => a.time);
                        svgs.push(displayBranches(nodeTree));
                        resolve();
                    })
            })(current, n));
        }
    }
    return importer.runAllPromises(promises)
        .then(() => children);
}

function gitTipOfTree(project) {
    svgs = [];
    return (Promise.resolve(project) || cloneProject(project, true))
        // TODO: git checkout command
        // TODO: pull all missing remmotes
        .then(r => {
            project = r;
            var nodeTree = _.sortBy(gitTree(project), a => a.time);
            const head = nodeTree.filter(r => r.branch.indexOf('HEAD') > -1)[0];
            const branch = head
                ? (nodeTree.filter(r => r.branch.indexOf(head.branch.replace(/HEAD *-*>* */ig, '').split(/\s*,\s*/ig)) > -1)[0] || nodeTree[0])
                : nodeTree[0];
            return displayBranches([branch]);
        })
        .then(r => svgs.push(r))
        //.then(() => resetAllBranches())
        .then(() => svgs)
}
module.exports = gitTipOfTree;

if(typeof $$ !== 'undefined') {
    var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
    var project = path.join(PROFILE_PATH, 'Documents/portal-copy-2018-1-24-23-49-46');
    $$.async();
    gitTipOfTree(project)
        .then(r => $$.html(r.join('')))
        .catch(e => $$.sendError(e))
}


Display graphs of different commands and scenarios using dry-run?

https://github.com/kpdecker/jsdiff

http://incaseofstairs.com/jsdiff/



In [ ]:
var path = require('path');
var gulp = require('gulp');
var run = require('gulp-run');
var tap = require('gulp-tap');
var importer = require('../Core');
var gulpPromise = importer.import('gulp task to a Promise', {gulp})

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = typeof project === 'undefined' ? path.join(PROFILE_PATH, 'Documents/portal') : project;

function getBranchBoundaries(p) {
    if(p) {
        project = p;
    }

    gulp.task('git watch', function () {
        return run('git diff -w a229417..498d5a5', {cwd: project})
            .pipe(tap(file => console.log(file)))
            .pipe(gulp.dest('output'))
    });

    return gulpPromise(['git watch'])
}
module.exports = getBranchBoundaries;

if(typeof $$ !== 'undefined') {
    $$.async();
    getBranchBoundaries(project)
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}



update git?


In [ ]:
var importer = require('../Core');
var createNewEvent = importer.import('create new calendar event');

function resetRebaseInstallEvent(project) {
    if(!project) {
        project = path.resolve(__dirname, '..', '..', 'jupytangular');
    }
    return createNewEvent('spawn child process', JSON.stringify({
        script: importer.interpret('git auto rebase').code + '\nnpm install',
        options: {cwd: project} // TODO: fix current working directory using project name?
    }, null, 4), {calendarId: 'aws'})
}
module.exports = resetRebaseInstallEvent;


github updates?

check github for commits since the current one?



In [ ]:
var _ = require('lodash');
var {JSDOM} = require('jsdom');
var importer = require('../Core');
var {
    gitTree, execCmd, getOauthClient, createNewEvent,
    updateEvent, cloneProject, getDaysEvents
} = importer.import([
    'd3.ipynb[format d3 tree]',
    'json git tree',
    'spawn child process',
    'import google calendar api',
    'create merge delete event',
    'git project directory',
    'days events',
    'create new calendar event'
]);

var options = {
    calendarId: 'aws'
}

// create a calendar event to retest this branch
function installBuildTestEvent(project) {
    return createNewEvent('spawn child process', JSON.stringify({
        script: `npm install\nnpm run build\nnpm run test`,
        options: {cwd: project} // TODO: fix current working directory using project name?
    }, null, 4), options)
}

function updateHeartbeatEvent(project, branch, commit) {
    return getOauthClient(options)
        .then(() => getDaysEvents(new Date(), options))
        .then(r => {
            const heartbeat = r.filter(r => r.event.summary === 'heartbeat'
                                       || r.event.summary === 'todays heartbeat items');
            try {
                var dom = new JSDOM('<body>' + (heartbeat[0].event.description || '').replace(/<br\/?>/igm, '\n') + '</body>');
                const desc = dom.window.document.body.textContent.split('\n');
                desc.forEach((c, i) => {
                    if(c.trim().length > 0 && c.includes('github updates')) {
                        const parameters = JSON.parse(((/[\{].*[\}]|[\[].*[\]]|(['"]).*\1/igm).exec(c) || ['""'])[0]);
                        if(parameters[0] === project && parameters[1] === branch) {
                            parameters[2] = commit;
                        }
                        desc[i] = 'github updates ' + JSON.stringify(parameters);
                    }
                });
                assert(desc.length, 'there is a serious problem with updating the commit');
                heartbeat[0].event.description = desc.join('\n');
                return updateEvent(heartbeat[0].event, options);
            } catch ( e ) {
                console.log(e);
            }
        })
}

function findBranchRemote(project, branch) {
    return execCmd(`git branch -r`, {cwd: project}) // 
        .then(r => {
            const remotes = r.join('\n').split(/\s*\n\s*/igm).filter(r => (r.split('/')[1] || '').trim() === branch);
            if(remotes.length === 0) {
                throw new Error('branch "' + branch + '" not found on any remotes ' + JSON.stringify(r))
            }
            remote = remotes[0].split('/')[0];
            return remote;
        })
        .then(() => ({remote, branch}))
}

function getUpdate(project, branch, fromCommit) {
    var projectCopy, latestCommit;
    return cloneProject(project)
        .then(p => {
            projectCopy = p;
            return findBranchRemote(projectCopy, branch);
        })
        .then(({remote, branch}) => execCmd(`
git fetch ${remote}
git log ${fromCommit || 'HEAD'}..${remote}/${branch}`, {cwd: projectCopy}))
        .then(log => {
            if(log.join('').trim().length > 0) {
                latestCommit = (/commit (.*)/ig).exec(log.join(''))[1];
                return cloneProject(projectCopy, true)
            } else {
                throw new Error('nothing for git to do');
            }
        })
        .then(r => {
            projectCopy = r;
            if(branch) {
                return execCmd(`
git reset --hard
git checkout ${branch}`, {cwd: projectCopy})
            }
        })
        .then(() => execCmd(importer.interpret('git auto rebase').code.trim()
                            + ` ${remote} ${branch}`, {cwd: projectCopy}))
        .then(() => installBuildTestEvent(projectCopy))
    // TODO: update heartbeat event description to pass in the latest commit
        .then(() => updateHeartbeatEvent(project, branch, latestCommit))
}
module.exports = getUpdate;

// git log currentbranch@{1}..currentbranch
// where the @{1} notation means "the commit the branch pointed to just before it last got updated".
// git log HEAD..swiftpage/development

if(typeof $$ !== 'undefined') {
    $$.async();
    getUpdate('portal')
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


TODO:

Rebase squash cherrypick automatically to create perfect feature branch?

Reset each file one at a time to make sure there aren't any unnecessary changes?

Use unit testing to warn about unnecessary modules?

Load relevant projects based on branch.  It could parse the .sln file and automatically patch whenever the checked out index changes.

Plot when a TODO: item is added and when it is removed on Google Calendar, then verify by checking commit -messages, remaining comments/function names with matching keywords (split by camel casing)?, plot dates completed for measurement of self set tasks and short term goals, which activities are harder than others?  better insight in to erroneous developer time estimates?

Interface for searching all TODO: items?  Calendar?



git project directory?

get fresh git project directory?


In [ ]:
var fs = require('fs');
var glob = require('glob');
var rimraf = require('rimraf');
var path = require('path');
var importer = require('../Core');
var listInProject = importer.import('list project files');

function deepCopy(project_orig) {
    const now = new Date();
    // delete old cloned directories
    const copies = glob.sync(path.basename(project_orig) + '-copy-*', {cwd: path.dirname(project_orig)});
    copies.forEach(c => {
        rimraf.sync(path.join(path.dirname(project_orig), c));
    });
    const projectName = path.basename(project_orig) + '-copy-' + now.getFullYear()
        + '-' + (now.getMonth() + 1) + '-' + now.getDate()
        + '-' + now.getHours() + '-' + now.getMinutes()
        + '-' + now.getSeconds();
    const project = path.join(path.dirname(project_orig), projectName);
    // do a deep copy of all git files
    console.log('cloning repo ' + project_orig + ' -> ' + project);
    return Promise.all([
        listInProject(project_orig, '**/*'),
        listInProject(project_orig + '/.git', '**/*'),
    ])
        // TODO: clone from remote
        .then(r => {
            for (const file of [].concat(...r).sort()) {
                const dirname = path.dirname(
                    file.replace(project_orig, project)).split(/\/|\\/ig);

                for (var i = 2; i <= dirname.length; i++) {
                    if (!fs.existsSync(dirname.slice(0, i).join('/'))) {
                        fs.mkdirSync(dirname.slice(0, i).join('/'));
                    }
                }
                fs.copyFileSync(file, file.replace(project_orig, project));
            }
        })
        .then(() => listInProject(project, '**/*.lock').forEach(p => rimraf.sync(p)))
        .then(() => project)
}

function cloneProject(project, copy) {
    var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
    if(!project) {
        throw new Error('specify project path or github');
    }
    if(fs.existsSync(path.resolve(project))) {
        project = path.resolve(project);
    } else if(fs.existsSync(path.join(PROFILE_PATH, 'Documents/' + project))) {
        project = path.resolve(path.join(PROFILE_PATH, 'Documents/' + project));
    } else if(fs.existsSync(path.join(PROFILE_PATH, project))) {
        project = path.resolve(path.join(PROFILE_PATH, project));
    } else if(project.match(/github.com\/[a-z0-9_-]+\/[a-z0-9_-]+/ig)) {
        // fetch remote
        const projectName = (/github.com\/([a-z0-9_-]+)\/([a-z0-9_-]+)/ig).exec(project)[2];
        const projectHttp = project.replace(/^(https*)*:*\/*\/*/ig, 'https://');
        if(fs.existsSync(path.join(PROFILE_PATH, projectName))) {
            project = path.resolve(path.join(PROFILE_PATH, projectName));
        } else {
            return execCmd(`
git clone -v ${projectHttp}
`, {cwd: path.resolve(PROFILE_PATH)})
                .then(r => path.resolve(path.join(PROFILE_PATH, projectName)))
                .then(r => copy ? deepCopy(r) : r)
        }
    } else {
        throw new Error('invalid project path "' + project + '"');
    }
    return Promise.resolve(project)
            .then(r => copy ? deepCopy(r) : r)
}
module.exports = cloneProject;



megamind?

github commit html acl?

TODO: parse html document and copy changes to commit


In [ ]:
var GitHubApi = require('github');
var ASTQ = require('astq');
var ASTY = require('asty');
var css = require('css');
var glob = require('glob');
var importer = require('../Core');
var loadScraped = importer.import('get scraped page');

// commit changes to github
function getGist(gist) {
    const github = new GitHubApi({
        host: 'api.github.com'
    });
    /*
    github.authenticate({
        type: 'basic',
        username: process.env.USERNAME,
        password: process.env.PASSWORD
    });
    */

    return github.gists.get({id: gist})
        .then(r => r.data.files);
}

function replaceSelectors(str, prefix) {
    const ast = css.parse(str);
    // TODO: add a check for media queries
    ast.stylesheet.rules.forEach(r => {
        if(typeof r.selectors === 'undefined') {
            return;
        }
        r.selectors.forEach((s, i) => {
            r.selectors[i] = prefix + ' ' + s;
        });
    })
    return css.stringify(ast);
}

// git 
function gitEditor(gist, file) {
    // TODO: use a Github repo as the input
    const HOST_ID = 'webtest123'
    gist = gist || 'act.com' || 'a572d0830ae72b962e12a57adaec7c52';
    file = file || 'index';
    return loadScraped(gist, file)
        .then(files => typeof files[ file + '-acl.json' ] !== 'undefined'
            ? Promise.resolve(files)
            : getGist(gist))
        .then(files => {
            // scan using an acl list, similar to easylist?
            // TODO: accept formats:
            //    {"selector": "selector"}
            //    {"glob-url@selector": "glob-template-path@selector"}
            //    {"selector": "html-file@selector"}
            //    {"selector": "html-file@xpath"} ?
            //    {"glob-file": {"glob-url"...} || ["selector"]}
            const paths = JSON.parse(files[ file + '-acl.json' ].content);
            const content = files[ file + '.html' ].content;
            const css = replaceSelectors(files[ file + '.css' ].content, '#' + HOST_ID);
            return {
                id: HOST_ID,
                html: '<style>' + css + '</style>' + content,
                acl: paths
            };
        })
}
module.exports = gitEditor;

if(typeof $$ !== 'undefined') {
    $$.async();
    gitEditor()
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


get scraped page?


In [ ]:
var {JSDOM} = require('jsdom');
var _ = require('lodash');
var glob = require('glob')
var path = require('path');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = path.join(PROFILE_PATH, 'Collections/crawls');

function noop() {}

function getAllLinks(doc) {
    var dom = new JSDOM(doc);
    var xpaths = importer.import('all elements xpath');
    xpaths.document = dom.window.document;
    xpaths.client = {
        execute: (func, ...args) => Promise.resolve({
            // TODO: Synchronous utility!?
            value: func.apply(dom.window.document, args)
        }),
        addCommand: noop
    };
    return xpaths.getAllXPath({
        links: '//a/@href|//*[@src]/@src',
        styles: '//link/@href|style/@text()'
    }); 
}

function fixLink(url, l) {
    return l.trim().match(/^\/\//ig)
       ? ('https:' + l.trim())
       : (l.trim().match(/^https?:\/\//ig)
         ? l.trim()
         : url.replace(/\/$/ig, '') + '/' + l.trim().replace(/^\//ig, ''));
}

function loadScraped(gist, file) {
    let files = {};
    const crawl = _.sortBy(
        glob.sync(
            '**/*' + gist.replace(/[^a-z0-9_-]/ig, '_') + '*',
            {cwd: project}
        ),
        a => fs.statSync(path.join(project, a)).mtime.getTime()
    ).reverse();
    // bail
    if(!crawl.length) {
        return files;
    }
    
    const entry = JSON.parse(fs.readFileSync(path.join(project, crawl[0])).toString());
    const content = entry.filter(r => r.url.includes(file)
                                 || file === 'index'
                                 && r.url.trim().replace(/https?:\/\/|\/$/ig, '').includes(gist))[0];
    if(content) {
        wrapXPath(content.content);
        const styles = content.styles.map(s => fixLink(content.url, s));
        const stylesheets = entry
            // TODO: fix this body doesn't work, remove <body tags from downloaded page
            .filter(r => styles.includes(r.url))
            .map(r => r.content.replace(/<body[^>]*?><pre[^>]*?>|<\/pre[^>]*?><\/body[^>]*?>/ig, ''))
            .filter(r => !r.match(/<[^>]*?>.*?<\/[^>]*?>/ig))
            .join('\n');
        files['index.html'] = {
            content: content.content
        };
        files['index-acl.json'] = {
            content: '[]'
        };
        files['index.css'] = {
            content: stylesheets
        };
    }
    return getAllLinks(content.content)
        .then(r => {})
        .then(() => files)
}
module.exports = loadScraped;


glob git branch?



In [ ]:
var glob = require('glob');
var minimatch = require('minimatch');
var importer = require('../Core');
var {
    cloneProject,
    execCmd
} = importer.import([
    'git project directory',
    'spawn child process'
]);

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = path.join(PROFILE_PATH, 'Documents/portal');

function globBranch(globs, project, remote, branch) {
    // TODO: not using default parameters because babel removes them entirely, fix RPC service using AST instead
    remote = remote || 'swiftpage';
    branch = branch || 'development';
    var projectCopy;
    return cloneProject(project, true)
        .then(p => {
            projectCopy = p;
            return execCmd(`
git fetch ${remote}
git log ${remote}/${branch}..${'HEAD'}`, {cwd: projectCopy});
        })
        .then(c => {
            commits = importer.regexToArray(/commit\s+(.*?)\s+/ig, c, 1);
            // TODO: match commits to specified branches
            console.log(commits[commits.length-1])
            return execCmd(`
git reset --mixed ${commits[commits.length-1]}
git status -s`, {cwd: projectCopy});
        })
        .then(r => {
            const files = r[1].split('\n').map(f => f.split(/\s+/ig)[2]).filter(f => f);
            var promises = [];
            Object.keys(globs).forEach(k => {
                const matches = files.filter(f => minimatch(f, k));
                if(matches.length) {
                    promises.push(resolve => execCmd(`
git checkout -B ${globs[k]}
git add ${matches.join('\ngit add ')}
git commit -m "auto-commit ${globs[k]}"
`, {cwd: projectCopy}).catch(e => console.log(e)).then(r => resolve(r)));
                }
            });
            return importer.runAllPromises(promises);
        })
        // TODO: add to commit message all commit messages / features
        // TODO: require minimatch for commit message <> feature
}
module.exports = globBranch;

/*
TODO:
	deleted:    .babelrc
	deleted:    .gitignore
	modified:   angular-cli.json
	modified:   config/ecommerceResources.ts
	modified:   config/environment.dev.ts
	modified:   config/identityResources.ts
	modified:   package-lock.json
	modified:   src/app/auth/forgot-password/forgot-password.component.html
	modified:   src/app/auth/forgot-password/forgot-password.component.spec.ts
	modified:   src/app/auth/forgot-password/forgot-password.component.ts
	modified:   src/app/auth/register-form/register-form.component.spec.ts
	modified:   src/app/auth/user.service.ts
	modified:   src/app/core/core.module.ts
	new file:   src/app/layout/custom-spinner/custom-spinner.component.html
	new file:   src/app/layout/custom-spinner/custom-spinner.component.scss
	new file:   src/app/layout/custom-spinner/custom-spinner.component.spec.ts
	new file:   src/app/layout/custom-spinner/custom-spinner.component.ts
	new file:   src/app/layout/custom-spinner/custom-spinner.module.ts
	modified:   src/app/layout/footer/footer.component.scss
	modified:   src/app/layout/layout.component.scss
	modified:   src/app/layout/layout.module.ts
	modified:   src/app/layout/menu/menu.component.scss
	modified:   src/app/layout/toolbar/toolbar.component.html
	deleted:    src/assets/.gitkeep
	deleted:    src/assets/.npmignore
	renamed:    src/assets/act-logo-circle.png -> src/assets/act-logo-circle.png
	renamed:    src/assets/act-pro.png -> src/assets/act-pro.png
	renamed:    src/assets/act_logo_small.png -> src/assets/act_logo_small.png
	renamed:    src/assets/actpremium.jpg -> src/assets/actpremium.jpg
	renamed:    src/assets/actpremiumcloud.png -> src/assets/actpremiumcloud.png
	renamed:    src/assets/aem_logo.png -> src/assets/aem_logo.png
	renamed:    src/assets/background.jpg -> src/assets/background.jpg
	renamed:    src/assets/colors.scss -> src/assets/colors.scss
	new file:   src/assets/css/three-dot-loader.scss
	renamed:    src/assets/hhc_logo.png -> src/assets/hhc_logo.png
	modified:   src/assets/i18n/en.json
	renamed:    src/assets/layout.scss -> src/assets/layout.scss
	renamed:    src/assets/loader.svg -> src/assets/loader.svg
	renamed:    src/assets/material-overrides.scss -> src/assets/material-overrides.scss
	renamed:    src/assets/noise.png -> src/assets/noise.png
	renamed:    src/assets/palette.scss -> src/assets/palette.scss
	modified:   src/index.html
	modified:   src/testing/account-stubs.ts
	modified:   src/testing/auth-stubs.ts


*/

if(typeof $$ !== 'undefined') {
    $$.async();
    globBranch({
        // TODO: pull from version1 api
        '**/+(custom-spinner|layout|core|assets|testing)/**': 'B-11801_Moving_Layout_To_Match_Mobile',
        '**/trial/**': 'B-11801_Loadding_Spinner',

        '**/auth/register*/**': 'B-11803_Portal_Register',
        '**/auth/login/**': 'B-11802_Portal_Login',
        '**/auth/forgot*/**': 'B-11802_Portal_Login_Missing_Or_Unscheduled_Work',
        
        '**/account/subscription/**': 'B-11819_Modify_Subscription',

        '**/*handheld*/**': 'B-11806_Addons',
        '**/*emarketing*/**': 'B-11804_AEM',
        '**/*support*/**': 'B-11807_Support',
        '**/*+(primary|premium)*/**': 'B-11818_Premium_Plus',
        '**/*billing*/**': 'B-11808_Billing',
        '**/*confirm*/**': 'B-11809_Review',

        '**/+(cart|order-tracker*|*line-item*)/**': 'B-11804_Cart',

        '**/account/overview/**': 'B-11812_Summary',
        '**/account/billing/**': 'B-11813_Billing',
        '**/account/subscription/**': 'B-11814_Manage_Subscription',
        '**/account/**': 'B-11811_My_Account',
    },
               // TODO: connect AST to git
               // ignore whitespace?
               // run all git commands and show the tree?
               project)
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


convert git history to google calendar events?


In [ ]:
var chrono = require('chrono-node');
var _ = require('lodash');
var importer = require('../Core');
var execCmd = importer.import('spawn child process');
var ISODateString = importer.import('convert date iso');
var correctCalendarId = importer.import('lookup calendar name');
var updateEvent = importer.import('update create merge event');
var offset = (new Date()).getTimezoneOffset() * 60 * 1000;

function gitRemote(project) {
    return execCmd(`
git remote -v
`, {cwd: project})
        .then(r => r[0].trim().split('\n').map(l => {
            var line = l.split('\t');
            return {
                name: line[0],
                address: line[1].split(' ')[0],
                function: line[1].split(' ')[1]
            };
        }))
}

function gitBranch(project, remote) {
    return execCmd(`
git remote show ${remote}
`, {cwd: project})
        .then(r => (/\s*HEAD branch:\s*(.*?)\s/ig).exec(r[0].trim())[1])
}

function gitLog(project, remote, branch) {
    return execCmd(`
git fetch ${remote}
git log --name-only --pretty=format:"%h%x09%an%x09%aD%x09%s" ${remote}/${branch}`, {cwd: project})
        .then(log => log[1].split('\n\n').map(l => {
            var fields = l.split('\n')[0].split('\t');
            var files = l.split('\n').slice(1);
            return {
                hash: fields[0],
                author: fields[1],
                date: chrono.parse(fields[2])[0].start.date(),
                message: fields[3],
                files: files
            }
        }))
}

function gitHistory(project, summary) {
    var remote, branch;
    if(typeof project == 'undefined') {
        project = '/Users/briancullinan/jupytangular2'
    }
    if(typeof summary == 'undefined') {
        summary = 'Worked on jupytangular'
    }
    return gitRemote(project)
        .then(remotes => remote = remotes.filter(r => r.function == '(fetch)')[0].name)
        .then(() => gitBranch(project, remote))
        .then(b => branch = b)
        .then(() => gitLog(project, remote, branch))
        .then(commits => {
            // group commits by a few hours and make a working event
            var grouped = _.groupBy(commits, c => {
                // round to the nearest 3 hours
                return Math.round(c.date.getTime() / (60 * 60 * 3 * 1000)) * 60 * 60 * 3 * 1000;
            })
            return Object.keys(grouped).map(t => {
                var min = _.min(grouped[t].map(g => g.date)).getTime();
                min = Math.round(min / (60 * 60 * 3 * 1000)) * 60 * 60 * 3 * 1000
                var max = _.max(grouped[t].map(g => g.date)).getTime();
                max = Math.round(max / (60 * 60 * 3 * 1000)) * 60 * 60 * 3 * 1000
                var s = grouped[t].map(g => g.message).filter(g => g != 'no message').join(', ')
                return {
                    start: {
                        dateTime: ISODateString(new Date(parseInt(min) - offset))
                    },
                    end: {
                        dateTime: ISODateString(new Date(parseInt(max) + 60 * 60 * 1000 - offset))
                    },
                    summary: summary + (s != '' ? (', ' + s) : ''),
                    description: JSON.stringify(grouped[t], null, 4)
                }
            })
        })
        .then(results => {
            return importer.runAllPromises(results
                .map(event => resolve => updateEvent(event, {})
                    .then(r => resolve(r))))
        })
}

module.exports = gitHistory;
